In [1]:
# --- Load Agents --- #
from agents.agent_reinforce.agent import REINFORCEAgent
from agents.agent_deepqn.agent import DeepQAgent
from agents.agent_ddpg.agent import DDPG_Agent

# --- Load Environments --- #
from environment.grid import GridEnv

# --- Load Necessary --- #
from collections import deque
import numpy as np
from utilities.helper import flatten


# --- Load Training --- #
from experiments.experiment import Experiment

%load_ext autoreload
%autoreload 2


C:\Users\danie\anaconda3\envs\drlnd\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


# Generalized

In [ ]:
environments = [GridEnv(num_agent = 2, agents_start = [(1,1)], goals_start=[(7,7)]), GridEnv(num_agent = 2)]
wrapped_agents = [[DeepQAgent(env.state_space, env.action_space.n) for n in range(env.num_agent)] for env in environments]
# wrapped_agents = [[DeepQAgent(env.state_space, env.action_space.n, dddpqagent_config) for n in range(env.num_agent)] for env in environments]

# dddpqagent_config=[{}, ]
# print(environments)
# print(wrapped_agents)


In [ ]:
ddpg_experiments=[Experiment("Test Experiment", env, agents, max_t=100, num_episodes=300) for agents, env in zip(wrapped_agents, environments)]

# dqn_experiments=[Experiment("Test Experiment", env, agents, max_t=100, num_episodes=300) for agents, env in zip(wrapped_agents, environments)]

# reinforce_experiments=[Experiment("Reinforce Experiment", env, agents, max_t=100, num_episodes=300) for agents, env in zip(wrapped_agents, environments)]

# print(experiments)

In [ ]:
for experiment in ddpg_experiments:
    score_history, state_history = experiment.run()
    experiment.save(score_history, state_history)

In [6]:
combination=np.array(np.meshgrid(num_players, agents_start))

In [14]:
#print(combination.T)
import itertools
print(len(list(itertools.product(*configs))))
# for r in itertools.product(num_players, agents_start):
#     for q in itertools.product(r, goals_start):
#         for s in itertools.product(q, prob_right_direction):
#             print(s)

72
